# 프로젝트 목표

게임 회사의 데이터 팀에 합류했을 때

다음 분기에 어떤 게임을 설계해야 할까라는 고민을 해결하기 위해, 

기존의 데이터에 기반하여 인사이트를 얻고, 의사결정을 하고자 하는 것



# 🕹️ Data Description

**Name** : 게임의 이름입니다.

**Platform** : 게임이 지원되는 플랫폼의 이름입니다.

**Year** : 게임이 출시된 연도입니다.

**Genre** : 게임의 장르입니다.

**Publisher** : 게임을 배급한 회사입니다.

**NA_Sales** : 북미지역에서의 출고량입니다.

**EU_Sales** : 유럽지역에서의 출고량입니다.

**JP_Sales** : 일본지역에서의 출고량입니다.

**Other_Sales** : 기타지역에서의 출고량입니다.

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [ ]:
df_original = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vgames2.csv")

In [ ]:
df_original.info()

# EDA



- [x] Year, Genre, Publisher column에 결측치 발견 -> 결측치 제거해주기

- [x] [Year] : float -> int 형변환 해야함 / 이상치 고쳐주기

- [x] Sales : object -> float 형변환 해야함
    - 기본 단위 M(1,000,000)이라고 가정
    - M 은 그냥 삭제하기
    - K 는 삭제하고 1000 곱하기

- [x] [Unnamed: 0] : column 삭제

In [ ]:
# Unnamed: 0 행 제거
df = df_original.drop('Unnamed: 0', axis=1)

In [ ]:
# 결측치 있는 행 제거
df = df.dropna(axis=0)

In [ ]:
# index 재정렬
df.reset_index(drop=True)

In [ ]:
# [Year] 형 변환
df['Year'] = df['Year'].astype(int)

In [ ]:
# 출시연도 부정확한 데이터들 존재
display(df[df['Year'] < 100])

In [ ]:
# 출시연도 바꿔주기

def year_error(i):
    if i < 50 :
        return i + 2000
    elif (i > 50 and i < 100):
        return i + 1900
    else:
        return i

df['Year'] = df['Year'].apply(year_error)

In [ ]:
# M 삭제
df['NA_Sales'] = df['NA_Sales'].str.replace('M', '')
df['EU_Sales'] = df['EU_Sales'].str.replace('M', '')
df['JP_Sales'] = df['JP_Sales'].str.replace('M', '')
df['Other_Sales'] = df['Other_Sales'].str.replace('M', '')

In [ ]:
# K 포함값들 바꿔주기

def fillzero (s):
    s.str.pad(width=5, side='left', fillchar='0')

In [ ]:
df['NA_Sales'] = df['NA_Sales'].str.pad(width=5, side='left', fillchar='0')
df['EU_Sales'] = df['EU_Sales'].str.pad(width=5, side='left', fillchar='0')
df['JP_Sales'] = df['JP_Sales'].str.pad(width=5, side='left', fillchar='0')
df['Other_Sales'] = df['Other_Sales'].str.pad(width=5, side='left', fillchar='0')

In [ ]:
df['NA_Sales'] = df['NA_Sales'].str.replace('K', '')
df['EU_Sales'] = df['EU_Sales'].str.replace('K', '')
df['JP_Sales'] = df['JP_Sales'].str.replace('K', '')
df['Other_Sales'] = df['Other_Sales'].str.replace('K', '')

In [ ]:
df['NA_Sales'] = df['NA_Sales'].str.replace('00000', '0.0')
df['EU_Sales'] = df['EU_Sales'].str.replace('00000', '0.0')
df['JP_Sales'] = df['JP_Sales'].str.replace('00000', '0.0')
df['Other_Sales'] = df['Other_Sales'].str.replace('00000', '0.0')

In [ ]:
df['NA_Sales'] = df['NA_Sales'].str.replace('0000', '0.0')
df['EU_Sales'] = df['EU_Sales'].str.replace('0000', '0.0')
df['JP_Sales'] = df['JP_Sales'].str.replace('0000', '0.0')
df['Other_Sales'] = df['Other_Sales'].str.replace('0000', '0.0')

In [ ]:
# 0450K 를 0.45M 으로 바꿔준다
def ktom(s):
    if '.' not in s :
        return s.replace("0", "0.", 1)
    return str(s)

In [ ]:
df['NA_Sales'] = df['NA_Sales'].apply(ktom)
df['EU_Sales'] = df['EU_Sales'].apply(ktom)
df['JP_Sales'] = df['JP_Sales'].apply(ktom)
df['Other_Sales'] = df['Other_Sales'].apply(ktom)

In [ ]:
# [Sales] float 형으로 변환
df['NA_Sales'] = df['NA_Sales'].astype(float)
df['EU_Sales'] = df['EU_Sales'].astype(float)
df['JP_Sales'] = df['JP_Sales'].astype(float)
df['Other_Sales'] = df['Other_Sales'].astype(float)